In [ ]:
import torch
from torch.utils.data import DataLoader
from tqdm.auto import tqdm
import numpy as np
from transformers import AutoProcessor
from pathlib import Path
from typing import Optional, Dict, Any, Tuple

from src.dataset import (
    CoSiRValidationDataset,
)
from src.model import CoSiRModel
from src.eval import EvaluationManager, EvaluationConfig, MetricResult
from src.utils import (
    FeatureManager,
    ExperimentManager,
    TrainableEmbeddingManager,
    get_representatives,
    get_umap,
)

In [ ]:
# 1. Basic Setup
from src.hook.eval_cosir import CoSiREvaluator
from src.utils import ExperimentManager, get_representatives, get_umap

In [ ]:
# 2. Load Experiment
experiment_path = (
    "/project/CoSiR/res/CoSiR_Experiment/coco/20250917_090002_CoSiR_Experiment"
)
evaluator = CoSiREvaluator(experiment_path)

In [ ]:
# 3. Load Components (handles config issues automatically)
feature_manager = evaluator.load_feature_manager()
embedding_manager = evaluator.load_embedding_manager()
sample_ids, all_embeddings = embedding_manager.get_all_embeddings()

In [ ]:

# 4. K-means Representatives (requires 2D embeddings)
num_reps = 20
representatives = get_representatives(all_embeddings, num_reps)
similarities = torch.mm(representatives, all_embeddings.T)

# 5. Arbitrary Embedding Retrieval
np.random.seed(42)
query_indices = np.random.choice(len(all_embeddings), size=10, replace=False)
query_embeddings = all_embeddings[query_indices]
retrieval_similarities = torch.mm(query_embeddings, all_embeddings.T)
top_k_similarities, top_k_indices = torch.topk(retrieval_similarities, k=20, dim=1)

# 6. UMAP Visualization
dummy_labels = np.arange(len(all_embeddings)) % 10
fig = get_umap(
    umap_features_np=all_embeddings.cpu().numpy(),
    umap_labels=dummy_labels,
    epoch=0,
    samples_to_track=[],
    z_threshold=3,
    no_outlier=True
)

Save Results

# 7. Create Experiment Context for Saving
exp_manager = ExperimentManager()
experiment_context = exp_manager.create_experiment(
    "my_analysis",
    description="CoSiR Analysis"
)

# 8. Save Results with Timestamps
experiment_context.save_artifact("representatives", representatives, "numpy", folder="results")
experiment_context.save_artifact("retrieval_results", results_dict, "pickle", folder="results")
experiment_context.save_artifact("umap_plot", fig, "figure", folder="plots")